# **Domain Specific Analysis**

## Objectives

*   Answer business requirement 1: 
    * The client is interested to understand the patterns between an airplanes design features and its Performance features, so that the client can learn which are the most relevant variables to consider when choosing **Engine Type** (jet, piston or propjet) in the design process of a new airplane.

## Inputs

* outputs/datasets/collection/airplane_performance_study.csv

## Outputs

* generate code that answers business requirement 1 and results that can be used to create airplane images with informative filenames that can be used to build Streamlit App ("Get to know the dataset"-page )

df_summary_stats






---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* Access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Imports

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import numpy as np

from matplotlib.lines import Line2D
from sklearn.linear_model import LinearRegression

# Load Data

* Dropping the columns with Meta Data 'Model' and 'Company' since these are identifier variables not needed for the study
* Dropping the two Engine "size" features 'THR' (Force with unit lbf) and 'SHP' (Power with unit SHP). The "THR" column is occupied (otherwise showing NaN) when the "Engine Type" is categorized with Jet and the same goes for the "SHP" when the "Engine Type" is categorized with piston or propjet. These two features () are interesting from an Aircraft Design perspective but since these are different quantities with different units they become difficult and awkward to compare with each other.

In [ ]:
df = pd.read_csv("/workspace/data-driven-design/outputs/datasets/collection/airplane_performance_study.csv")
df.head(10)


---

# Data Exploration from the domain specific perspective of Airplane Design and Performance

## Mean and extremes in the data set

We are interested to get more familiar with the dataset and to see if we can pin which airplanes that constitutes the most average and extremes in the data set. The Categorical feature Engine are not considered in this exploratory exercise.

Acclaim Ultra (M20V) (Mooney Aircraft) MIN: 280.0 is selected for the min value for THR - feature. Mighty strange since it has a prop. Probable explanation is that this value still exxist for this airplane (check!). This airplane is with a Vmax of 242 knots the fastest single-engine production piston aircraft in the world. The Engine Type and Multi_Engine for this airplane, I just discovered (Vmax is however correct) is incorrect. It should be a Engine Type: Piston and Multi Engine: false and these wrong entries is the reason for why it appears in the min THR category. I have NOT corrected this error yet.  

In [ ]:
# Assuming df is defined and contains the relevant features and company/model columns
# Create an empty DataFrame with columns showing what I want to display
columns = ['FEATURE', 'units', 'MIN_model', 'MIN_company', 'MIN_value', 
           'MEAN_model', 'MEAN_company', 'MEAN_value', 
           'MAX_model', 'MAX_company', 'MAX_value']
df_summary_stats = pd.DataFrame(columns=columns)

# Fill the 'FEATURE' column with all relevant features leaving the categorical values out
df_summary_stats['FEATURE'] = ['Wing_Span', 'Length', 'Height', 
                                'THR', 'SHP', 'AUW', 'MEW',
                                'FW', 'Vmax', 'Vcruise', 'Vstall',
                                'Range', 'Hmax', 'Hmax_(One)',
                                'ROC', 'ROC_(One)', 'Vlo', 'Slo',
                                'Vl', 'Sl']

# Define units for each feature
units = ['ft', 'ft', 'ft',
         'lbf', 'HP',
         'lb', 'lb', 'lb', 'knots',
         'knots', 'knots', 'N.m.',
         'ft', 'ft', 'ft/min', 'ft/min',
         'ft', 'ft/min', 'ft', 'ft/min']

# Add the units to the new column
df_summary_stats['units'] = units

# Calculate statistics and fill df_summary_stats
for feature in df_summary_stats['FEATURE']:
    if feature in df.columns:
        # Minimum statistics
        min_value = df[feature].min()
        min_index = df[feature].idxmin()
        df_summary_stats.loc[df_summary_stats['FEATURE'] == feature, 'MIN_model'] = df.loc[min_index, 'Model']
        df_summary_stats.loc[df_summary_stats['FEATURE'] == feature, 'MIN_company'] = df.loc[min_index, 'Company']
        df_summary_stats.loc[df_summary_stats['FEATURE'] == feature, 'MIN_value'] = f"{min_value:.1f}"  # Format to 1 decimal

        # Mean statistics
        mean_value = df[feature].mean()
        df_summary_stats.loc[df_summary_stats['FEATURE'] == feature, 'MEAN_value'] = f"{mean_value:.1f}"  # Format to 1 decimal
        
        # Finding the closest value to the mean
        closest_index = (df[feature] - mean_value).abs().idxmin()
        df_summary_stats.loc[df_summary_stats['FEATURE'] == feature, 'MEAN_model'] = df.loc[closest_index, 'Model']
        df_summary_stats.loc[df_summary_stats['FEATURE'] == feature, 'MEAN_company'] = df.loc[closest_index, 'Company']

        # Maximum statistics
        max_value = df[feature].max()
        max_index = df[feature].idxmax()
        df_summary_stats.loc[df_summary_stats['FEATURE'] == feature, 'MAX_model'] = df.loc[max_index, 'Model']
        df_summary_stats.loc[df_summary_stats['FEATURE'] == feature, 'MAX_company'] = df.loc[max_index, 'Company']
        df_summary_stats.loc[df_summary_stats['FEATURE'] == feature, 'MAX_value'] = f"{max_value:.1f}"  # Format to 1 decimal
    else:
        print(f"Feature '{feature}' not found in the DataFrame.")

df_summary_stats


In [7]:
df_summary_stats.to_csv('/workspace/data-driven-design/outputs/datasets/collection/df_summary_stats.csv', index=False)

In [ ]:
df_summary_stats


---

## Range dependencie on proportion of fuel Weight

The Range Equation (derived from the Breguet equations) include the proportion of the fuel weight to the overall weight of the aircraft. It measures this by the relationship between the initial weight of the aircraft (before take off) to the weight of the airplane (after landing) after the fuel has been burned off.
A very interesting pattern emerges where it is not linear but also not exponential, rather a sort of two-phased linearity with one slope up until a 'Wi/Wf' 1.35 after which the slope changes abruptly. Perhaps the reason for this suprise pattern lies in the the Engine types. This would warrant the same plot to be printed with the different Engine types separately!

<img src="/workspace/data-driven-design/images_notebook/range_eq.png" alt="Range Equationr" height="300" />

We have manually identified (by scrolling the Wi/Wf column) and removed the outlier that must be due to erroniouos data in FW and/or AUW.

In [ ]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

print(df)

In [ ]:
# Print features for data point in row 467
data_point = df.iloc[467]
print(data_point)

In [12]:
# Remove extreme outlier (most likely due to erronious data) from the DataFrameat index 467.
df = df.drop(index=467).reset_index(drop=True)

In [ ]:
    # Remove extreme outlier from the DataFrame
    df = df.drop(index=467).reset_index(drop=True)

    # Calculate Wi / Wf based on the formula
    df['Wi/Wf'] = df['AUW'] / (df['AUW'] - df['FW'])

    # Create a regression plot using seaborn
    plt.figure(figsize=(10, 6))

    # Linear regression with orange scatter points
    sns.regplot(x='Wi/Wf', y='Range', data=df, label='Linear', color='blue', 
                scatter_kws={'color': 'orange'})

    # Quadratic regression
    sns.regplot(x='Wi/Wf', y='Range', data=df, order=2, label='Quadratic', color='red', 
                scatter_kws={'color': 'orange'})  # Keep scatter orange for both

    # Customize the plot
    plt.title('Effect of Wi/Wf on Range')
    plt.xlabel('Wi/Wf')
    plt.ylabel('Range')
    plt.grid()

    # Create custom legend handles
    custom_legend = [
        Line2D([0], [0], color='blue', label='Linear Regression'),
        Line2D([0], [0], color='red', label='Quadratic Regression')
    ]
    plt.legend(handles=custom_legend)

    # Display the plot
    plt.show()

---

## Ceiling as a function of Wing Span

Finally, my former prof. Peter Lissaman found, when developing the solar powered High Altitude Airplane Helios (Wing Span: 75 m) at Aerovironment/NASA, that Hmax was completely dependent on Wing Span. Since this is not obvious from the equations of Hmax it will be interesting to see if this also applies to General Aviation. We can see that the pattern from Helios is reproducible also for General Aviation (at least for 2 out of 3 Engine types). See Airplane Performance Predictor Streamlit app for more conclusions.

<br>
<img src="/workspace/data-driven-design/images_notebook/helios.jpg" alt="Equation for" style="width: 40%;"/>

In [ ]:
# Set the style for the plot
sns.set(style="whitegrid")

# Create the plot
plt.figure(figsize=(10, 6))

# Plot regression line for Piston (Engine_Type = 0)
sns.regplot(x='Wing_Span', y='Hmax', data=df[df['Engine_Type'] == 0], 
            scatter_kws={'alpha': 0.5}, 
            line_kws={'label': 'Piston'}, 
            color='blue')

# Plot regression line for Propjet (Engine_Type = 1)
sns.regplot(x='Wing_Span', y='Hmax', data=df[df['Engine_Type'] == 1], 
            scatter_kws={'alpha': 0.5}, 
            line_kws={'label': 'Propjet'}, 
            color='orange')

# Plot regression line for Jet (Engine_Type = 2)
sns.regplot(x='Wing_Span', y='Hmax', data=df[df['Engine_Type'] == 2], 
            scatter_kws={'alpha': 0.5}, 
            line_kws={'label': 'Jet'}, 
            color='green')

# Add legend and titles
plt.legend()
plt.title('Effect of Wing Span on Hmax by Engine Type')
plt.xlabel('Wing Span')
plt.ylabel('Hmax')

# Plot
plt.show()


---

## Exploration of 3D Regression plots

Below is an exercise to get familiar with the 3D plots with a regression surface instead of a regression line.

In [ ]:
import pandas as pd
import plotly.graph_objs as go
import numpy as np

# Assuming df is your DataFrame with 'AUW', 'FW', and 'Range' columns

# Create a grid for AUW and FW using the unique values
AUW_range = df['AUW'].unique()
FW_range = df['FW'].unique()

# Create a meshgrid
AUW_grid, FW_grid = np.meshgrid(AUW_range, FW_range)

# Create a pivot table for Z values
Z = df.pivot_table(values='Range', index='FW', columns='AUW').values

# Create the figure
fig = go.Figure(data=[go.Surface(z=Z, x=AUW_grid, y=FW_grid)])
fig.update_layout(title='3D Surface Plot of AUW vs. FW',
                  scene=dict(xaxis_title='AUW',
                             yaxis_title='FW',
                             zaxis_title='Range'),
                  autosize=True)

# Show the plot
fig.show()


In [ ]:
# Assuming df is your DataFrame with 'AUW', 'FW', and 'Range' columns

# Prepare the data
X = df[['AUW', 'FW']]
y = df['Range']

# Fit the linear regression model
model = LinearRegression()
model.fit(X, y)

# Create a grid for AUW and FW using the unique values
AUW_range = np.linspace(df['AUW'].min(), df['AUW'].max(), 100)
FW_range = np.linspace(df['FW'].min(), df['FW'].max(), 100)
AUW_grid, FW_grid = np.meshgrid(AUW_range, FW_range)

# Predict Z values using the regression model
Z = model.predict(np.c_[AUW_grid.ravel(), FW_grid.ravel()]).reshape(AUW_grid.shape)

# Create the figure
fig = go.Figure()

# Add the regression plane
fig.add_trace(go.Surface(z=Z, x=AUW_grid, y=FW_grid, colorscale='Viridis', opacity=0.5, name='Regression Plane'))

# Add the scatter plot
fig.add_trace(go.Scatter3d(x=df['AUW'], y=df['FW'], z=df['Range'], mode='markers', 
                             marker=dict(size=5, color='red', opacity=0.8), name='Data Points'))

# Update layout
fig.update_layout(title='3D Regression Plane with Scatter Plot',
                  scene=dict(xaxis_title='AUW',
                             yaxis_title='FW',
                             zaxis_title='Range'),
                  autosize=True)

# Show the plot
fig.show()
